In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [2]:
import tensorflow as tf

In [3]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
#   try:
#     tf.config.set_visible_devices(gpus[0], 'GPU')
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
#   except RuntimeError as e:
#     # Visible devices must be set before GPUs have been initialized
#     print(e)


In [4]:
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   try:
#     # Currently, memory growth needs to be the same across GPUs
#     for gpu in gpus:
#       tf.config.experimental.set_memory_growth(gpu, True)
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
#   except RuntimeError as e:
#     # Memory growth must be set before GPUs have been initialized
#     print(e)


In [5]:
import re

In [6]:
import sys

In [7]:
from tensorflow import keras

In [8]:
filename = "bad_luck_brian_memetext.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
raw_text = raw_text.replace('\n',' ')

In [9]:
print(raw_text)

"i'm on a boat!"|titanic drinks fresh milk out of a cow|it was a bull ringtone goes of at funeral|highway to hell get touched by jesus|in a mexican prison gets hit by car sees a bright light|second car yolo|reincarnated hot girl gets in the elevator|farted 2 seconds ago googles 'gary oldman' forgets the 'r'|scarred for life steps outside to see what all the 'buzz' is about|africanized bees wishes upon a star|gets hit by meteor goes hunting|gets shot by a deer commits suicide|gets arrested gets a new follower on twitter|slenderman sends mom friend request on facebook|denied qualifies for olympic table tennis|plays china in first round at girlfriends house for the first time|takes a dump and clogs toilet is batman for halloween|parents die throws rock at the ground|misses kids on call of duty say they slept with his mom|true wins a free ride on a ship|titanic  gets a pet|grumpy cat orders happy meal|gets sad meal goes to jail in monopoly|gets raped throws a party|not invited mom hires ma

In [10]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [11]:
char_to_int

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 "'": 4,
 ',': 5,
 '-': 6,
 '.': 7,
 '/': 8,
 '0': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 '4': 13,
 '5': 14,
 '6': 15,
 '7': 16,
 '8': 17,
 '9': 18,
 'a': 19,
 'b': 20,
 'c': 21,
 'd': 22,
 'e': 23,
 'f': 24,
 'g': 25,
 'h': 26,
 'i': 27,
 'j': 28,
 'k': 29,
 'l': 30,
 'm': 31,
 'n': 32,
 'o': 33,
 'p': 34,
 'q': 35,
 'r': 36,
 's': 37,
 't': 38,
 'u': 39,
 'v': 40,
 'w': 41,
 'x': 42,
 'y': 43,
 'z': 44,
 '|': 45}

In [12]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  13468
Total Vocab:  46


In [13]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  13368


In [14]:
print(seq_in)

the author eats a jawbreaker |breaks jaw rich relative left him millions |so says the lawyers email 


In [15]:
print(dataX[:5])

[[2, 27, 4, 31, 0, 33, 32, 0, 19, 0, 20, 33, 19, 38, 1, 2, 45, 38, 27, 38, 19, 32, 27, 21, 0, 22, 36, 27, 32, 29, 37, 0, 24, 36, 23, 37, 26, 0, 31, 27, 30, 29, 0, 33, 39, 38, 0, 33, 24, 0, 19, 0, 21, 33, 41, 45, 27, 38, 0, 41, 19, 37, 0, 19, 0, 20, 39, 30, 30, 0, 36, 27, 32, 25, 38, 33, 32, 23, 0, 25, 33, 23, 37, 0, 33, 24, 0, 19, 38, 0, 24, 39, 32, 23, 36, 19, 30, 45, 26, 27], [27, 4, 31, 0, 33, 32, 0, 19, 0, 20, 33, 19, 38, 1, 2, 45, 38, 27, 38, 19, 32, 27, 21, 0, 22, 36, 27, 32, 29, 37, 0, 24, 36, 23, 37, 26, 0, 31, 27, 30, 29, 0, 33, 39, 38, 0, 33, 24, 0, 19, 0, 21, 33, 41, 45, 27, 38, 0, 41, 19, 37, 0, 19, 0, 20, 39, 30, 30, 0, 36, 27, 32, 25, 38, 33, 32, 23, 0, 25, 33, 23, 37, 0, 33, 24, 0, 19, 38, 0, 24, 39, 32, 23, 36, 19, 30, 45, 26, 27, 25], [4, 31, 0, 33, 32, 0, 19, 0, 20, 33, 19, 38, 1, 2, 45, 38, 27, 38, 19, 32, 27, 21, 0, 22, 36, 27, 32, 29, 37, 0, 24, 36, 23, 37, 26, 0, 31, 27, 30, 29, 0, 33, 39, 38, 0, 33, 24, 0, 19, 0, 21, 33, 41, 45, 27, 38, 0, 41, 19, 37, 0, 19, 0, 2

In [16]:
print(dataY[:5])

[25, 26, 41, 19, 43]


In [17]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [18]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [20]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 256)          264192    
_________________________________________________________________
dropout (Dropout)            (None, 100, 256)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 46)                11822     
Total params: 801,326
Trainable params: 801,326
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
model.fit(X[:500000], y[:500000], epochs=100, batch_size=128)
# [:500000]
model.save("v5_l")

NameError: name 'model' is not defined

In [32]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [33]:
start = numpy.random.randint(0, len(dataX)-1)
# start = "hehe "
pattern = dataX[start]
# pattern = "hehe "
print("Seed:")
# print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
Done.


In [21]:
model2 = keras.models.load_model("v3")

In [37]:
# start = numpy.random.randint(0, len(dataX)-1)
# pattern = dataX[start]
word = "accidently "
pattern = [char_to_int[char] for char in word]
# pattern2 = [int_to_char[char] for char in patterns]
# print(pattern2)
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(200):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model2.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" accidently  "
lakes ayea say ho pooked fotl rake holategon  teet of honeng fiel gor tcf 2l"ageu gio so fofeas hieo on deme thtter wines wh dovn ses  oeat oo cone thieer aidce so deiahi fites foono goe bin gies coe 
Done.


In [ ]:
[char_to_int[char] for char in seq_in]

In [ ]:
a = result2.split("|")

In [97]:
a

[' hires magician for birthday party',
 'voldemort  takes a bullet for a girl',
 'gets friendzoned dies in video game',
 'fifa  deletes e-mail offering millyooms ofro fall',
 'oon oerear first a party',
 'gets kicked out w']

In [87]:
seq_in

['s',
 't',
 'e',
 'r',
 ' ',
 'g',
 'i',
 'r',
 'l',
 ' ',
 'b',
 'u',
 'r',
 'n',
 's',
 ' ',
 'i',
 't',
 ' ',
 't',
 'o',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'g',
 'r',
 'o',
 'u',
 'n',
 'd',
 ' ',
 'f',
 'i',
 'n',
 'a',
 'l',
 'l',
 'y',
 ' ',
 'g',
 'e',
 't',
 's',
 ' ',
 'a',
 ' ',
 'g',
 'i',
 'r',
 'l',
 'f',
 'r',
 'i',
 'e',
 'n',
 'd',
 '|',
 'i',
 't',
 's',
 ' ',
 'j',
 'u',
 's',
 't',
 'i',
 'n',
 ' ',
 'b',
 'i',
 'e',
 'b',
 'e',
 'r',
 ' ',
 'h',
 'i',
 ' ',
 'p',
 'i',
 'e',
 'p',
 'l',
 'e',
 's',
 '|',
 'f',
 'r',
 'e',
 'n',
 'd',
 ' ',
 's',
 'h',
 'e',
 ' ',
 'h',
 'e']